<a href="https://colab.research.google.com/github/tkasod2/TIL/blob/main/sqlite/sqlite3%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3

In [2]:
print(sqlite3.sqlite_version)
print(sqlite3.version)

3.22.0
2.6.0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dbpath = "/content/drive/MyDrive/sql/sample.db"
conn = sqlite3.connect(dbpath)

In [5]:
cur = conn.cursor()

# cur.execute(하나의 SQL문~~)
# cur.executescript(하나 이상의 SQL문)
# cur.executemany(~~~, ~~~) --> Data만 바꿔가면서 동일한 SQL문을 반복시행

# * SELECT문 실행 후 적용 진행
# cur.fetchone() --> 하나의 행을 불러옴(출력)
# cut.fetchall() --> 모든 행을 불러옴(출력)

In [6]:
script = """
-- 1) employees 테이블이 이미 있다면 제거
DROP TABLE IF EXISTS employees;

-- 2) employees 테이블 생성 
-- 필드명, (필드)데이터 타입, 제약조건(constraints ;ex. AUTOINCREMENT, NOT NULL)
CREATE TABLE employees( 
id INTEGER PRIMARY KEY AUTOINCREMENT, --숫자를 저장, 이 테이블의 주요 Key(기본 키)로 지정, 값을 따로 입력하지 않으면 자동 증가 숫자 부여
name TEXT NOT NULL, --빈 값이 저장되는 것을 허용하지 않음
salary REAL, --소수점이 들어간 자료형
department TEXT,
position TEXT,
hireDate TEXT);

-- 3) employees 테이블에 데이터(row == record == instance) 넣기
-- Format) INSERT INTO 테이블명(필드명, 필드명, ...) VALUES(실제값, 실제값, ...) <- 지정한 필드의 수와 넣어주는 값의 갯수가 동일해야 함

INSERT INTO employees(name, salary, department, position, hireDate) VALUES('Dave', 300, 'Marketing', 'LV1', '2020-01-01');
INSERT INTO employees(name, salary, department, position, hireDate) VALUES('Clara', 420, 'Sales', 'LV2', '2018-01-11');
INSERT INTO employees(id, name, salary, department, position, hireDate) VALUES(3, 'Jane', 620, 'Developer', 'LV4', '2015-11-01');

-- 전체 필드에 빠짐없이 값을 넣을 시 필드명 생략 가능 
INSERT INTO employees VALUES(4, 'Peter', 530, 'Developer', 'LV2', '2020-11-01'); 
"""

cur.executescript(script) 
conn.commit()

In [7]:
data = [('Elena', 510, 'Recruiter', 'LV3', '2020-07-01'), 
        ('Sujan', 710, 'HR', 'LV5', '2014-06-01'),
        ('Jake', 210, 'CEO', 'LV8', '2012-01-01')]

cur.executemany("INSERT INTO employees(name, salary, department, position, hireDate)\
                VALUES(?, ?, ?, ?, ?);", data)
conn.commit()

In [10]:
cur.execute("SELECT * FROM employees;")
employee_list = cur.fetchall()
for employee in employee_list:
    print(employee)

(1, 'Dave', 300.0, 'Marketing', 'LV1', '2020-01-01')
(2, 'Clara', 420.0, 'Sales', 'LV2', '2018-01-11')
(3, 'Jane', 620.0, 'Developer', 'LV4', '2015-11-01')
(4, 'Peter', 530.0, 'Developer', 'LV2', '2020-11-01')
(5, 'Elena', 510.0, 'Recruiter', 'LV3', '2020-07-01')
(6, 'Sujan', 710.0, 'HR', 'LV5', '2014-06-01')
(7, 'Jake', 210.0, 'CEO', 'LV8', '2012-01-01')


In [13]:
print([ col[0] for col in cur.description ])

['id', 'name', 'salary', 'department', 'position', 'hireDate']


In [14]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM employees;", conn) 
df

,id,name,salary,department,position,hireDate
0,1,Dave,300.0,Marketing,LV1,2020-01-01
1,2,Clara,420.0,Sales,LV2,2018-01-11
2,3,Jane,620.0,Developer,LV4,2015-11-01
3,4,Peter,530.0,Developer,LV2,2020-11-01
4,5,Elena,510.0,Recruiter,LV3,2020-07-01
5,6,Sujan,710.0,HR,LV5,2014-06-01
6,7,Jake,210.0,CEO,LV8,2012-01-01


In [15]:
conn.close()

## 1) Pandas <-> SQLite3

In [43]:
a = (1051,1051,1051,1051,1051,2073,2073,2073,2073,3063,3063,3063,3053,3024,3024)
b = ('aBxD','aBxD','aBxD','aBxD','aBxD','bDbD','bDbD','bDbC','bDbC','aaaa','aaab','aaac', '5DaD','aDDD','aDDE')
c = ('2021-10-31','2021-10-31','2021-10-31','2021-11-01','2021-11-01','2021-10-31','2021-10-31','2021-10-30', '2021-10-29','2021-11-07','2021-11-08','2021-11-09','2021-11-09','2021-11-09','2021-11-10')
df = pd.DataFrame({"user_id":a, "session_id":b, "date": c})
df

,user_id,session_id,date
0,1051,aBxD,2021-10-31
1,1051,aBxD,2021-10-31
2,1051,aBxD,2021-10-31
3,1051,aBxD,2021-11-01
4,1051,aBxD,2021-11-01
5,2073,bDbD,2021-10-31
6,2073,bDbD,2021-10-31
7,2073,bDbC,2021-10-30
8,2073,bDbC,2021-10-29
9,3063,aaaa,2021-11-07


In [23]:
dbpath = "/content/drive/MyDrive/sql/user.db"
conn = sqlite3.connect(dbpath)

In [25]:
# Save
df.to_sql('user',conn)

In [42]:
# Load
df = pd.read_sql('SELECT * FROM user;', conn, index_col = 'index')
df.stack().unstack()

,user_id,session_id,date
index,,,
0,1051,aBxD,2021-10-31
1,1051,aBxD,2021-10-31
2,1051,aBxD,2021-10-31
3,1051,aBxD,2021-11-01
4,1051,aBxD,2021-11-01
5,2073,bDbD,2021-10-31
6,2073,bDbD,2021-10-31
7,2073,bDbC,2021-10-30
8,2073,bDbC,2021-10-29


In [34]:
cur = conn.cursor()

cur.execute('''
SELECT * FROM user
''')
# cur.executescript(하나 이상의 SQL문)
# cur.executemany(~~~, ~~~) --> Data만 바꿔가면서 동일한 SQL문을 반복시행

# * SELECT문 실행 후 적용 진행
# cur.fetchone() --> 하나의 행을 불러옴(출력)
# cut.fetchall() --> 모든 행을 불러옴(출력)

cur.fetchall()

[(0, 1051, 'aBxD', '2021-10-31'),
 (1, 1051, 'aBxD', '2021-10-31'),
 (2, 1051, 'aBxD', '2021-10-31'),
 (3, 1051, 'aBxD', '2021-11-01'),
 (4, 1051, 'aBxD', '2021-11-01'),
 (5, 2073, 'bDbD', '2021-10-31'),
 (6, 2073, 'bDbD', '2021-10-31'),
 (7, 2073, 'bDbC', '2021-10-30'),
 (8, 2073, 'bDbC', '2021-10-29'),
 (9, 3063, 'aaaa', '2021-11-07'),
 (10, 3063, 'aaab', '2021-11-08'),
 (11, 3063, 'aaac', '2021-11-09'),
 (12, 3053, '5DaD', '2021-11-09'),
 (13, 3024, 'aDDD', '2021-11-09'),
 (14, 3024, 'aDDE', '2021-11-10')]